# Node Embeddings

<b> Recap </b>

Previously we discussed traditional feature based methods for graph ML, where given a graph G = (V,E) we wanted to extract node, link and graph-level features and learn a model (svm, logistic regressor, etc.) that maps features to labels.

Input Graph -> Feature Engineering -> Learning Algorithm -> Prediction

<b> Now </b>

With <b>Representation Learning</b> we automatically learn the features, without needing to manually construct the features as we did before (feature engineering). 

<b> Goal </b>

Efficient task-independent feature learning for machine learning with graphs

## Why Embeddings?

<b>Task:</b> map nodes into an embedding space

* Similarity of embeddings between nodes indicates their similarity in the network.
* Encode network information
* Potentially used for many downstream prediction tasks such as node classification, link prediction, graph classification, anomalous node detection, clustering, etc.

## Embedding Nodes

<b>Goal:</b> Find an encoder function such that, when nodes u and v are encoded to the embedding space, the following assumption is true:
the similarity between nodes u and v in the network is close to the similarity of the embedding representation of u and v in embedding space.

similarity(u,v) ~ z_v.T @ z_u

1. Encoder maps nodes to embeddings: Encoder(u) -> z_u.
2. Define a similarity function for the original network.
3. Decoder maps from embeddings to the embedding similarity score: dot preduct.
4. Optimize the parameters such that similarity(u,v) ~ decoder(encoder(u), encoder(v)).

